# Understanding the Viterbi Algorithm for DNA Sequence Analysis

## Introduction
In our BEE-102 class, we've been studying this cool algorithm called Viterbi that helps us figure out hidden patterns in DNA sequences. It's basically used to find the most likely sequence of hidden states (like whether a part of DNA is an exon or intron) based on the nucleotide sequence we can observe. It's pretty neat once you get the hang of it!

## What's a Hidden Markov Model?
Before diving into Viterbi, I should explain what a Hidden Markov Model (HMM) is. It's like having two parallel processes:
- One process that we can't directly see (the hidden states)
- Another process that we can observe (the emissions)

In DNA analysis:
- The hidden states might be things like exons (E), introns (I), and splice sites (5)
- What we actually observe are just the nucleotides (A, T, G, C)

In [7]:
hmm_states = ['E', '5', 'I']
dna_bases = ['A', 'C', 'G', 'T']

start_probs = {'E': 1.0, '5': 0.0, 'I': 0.0}

trans_probs = {
    'Start': {'E':1.0, '5': 0.0, 'I': 0.0, 'End': 0.0},
    'E': {'E': 0.9, '5': 0.1 , 'I': 0.0, 'End': 0.0},
    '5': {'E': 0.0, '5': 0.0 , 'I': 1.0, 'End': 0.0},
    'I': {'E': 0.0, '5': 0.0 , 'I': 0.9, 'End': 0.1}
}

emit_probs = {
    'E': {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25},
    '5': {'A': 0.05, 'C': 0.0, 'G': 0.95, 'T': 0.0},
    'I': {'A': 0.4, 'C': 0.1, 'G': 0.1, 'T': 0.4},
}

## The Problem We're Solving
We have a DNA sequence like "CTTCATGTGAAAGCAGACGTAAGTCA", and we want to know which parts are most likely exons, introns, or splice sites. But we can't directly see these features - we have to infer them from the patterns of nucleotides.

## How Viterbi Works
The Viterbi algorithm is all about finding the most likely path through hidden states. It works by:

1. Starting from the beginning of the sequence
2. For each position, calculating the most likely state based on:
   - What we've seen so far
   - The transition probabilities between states
   - The emission probabilities of observing particular nucleotides

The cool thing is it doesn't have to try every possible path (which would be CRAZY for long sequences) - it uses dynamic programming to be efficient.

In [8]:
import math
def safe_log(num):
    if (num == 0):
        return -math.inf
    else:
        return math.log(num)

def calc_path_probability(states, sequence):
    if len(states) != len(sequence):
        raise ValueError("States and sequence must be the same length.")

    total_log_prob = 0.0
    last_state = 'Start'

    # Main logic: Calculate the probability by adding log probabilities of transitions and emissions
    for curr_state, base in zip(states, sequence):
        trans_prob = safe_log(trans_probs[last_state][curr_state])
        emit_prob = safe_log(emit_probs[curr_state][base])
        total_log_prob += trans_prob + emit_prob
        last_state = curr_state
    
    # Add final transition to End state if we're in an intron
    if last_state == 'I':
        total_log_prob += safe_log(trans_probs[last_state]['End'])

    return total_log_prob

print(round(calc_path_probability("EEEEEEEEEEEEEEEEEE5IIIIIII", "CTTCATGTGAAAGCAGACGTAAGTCA"), 2))

-41.22


## Key Components for Our Implementation

### 1. States
In our model, we have three states:
- E: Exon regions (coding DNA)
- I: Intron regions (non-coding DNA) 
- 5: 5' splice site (transition region between exon and intron)

### 2. Transition Matrix
This shows the probability of moving from one state to another:
- P(E→E): How likely we stay in an exon
- P(E→5): How likely we transition from exon to splice site
- P(5→I): How likely we go from splice site to intron
- P(I→I): How likely we stay in an intron
- etc.

### 3. Emission Matrix
This shows how likely each state is to emit particular nucleotides:
- P(A|E): Probability of seeing an A in an exon
- P(G|I): Probability of seeing a G in an intron
- etc.

### 4. Using Log Probabilities
One tricky part is that when we multiply many small probabilities, we can get numerical underflow. To avoid this, we use log probabilities. Adding log probabilities is equivalent to multiplying regular probabilities.

In [9]:
def find_best_path(dna_seq):
    seq_len = len(dna_seq)
    
    dp_table = [{} for _ in range(seq_len)] # Stores probabilities at each step
    backtrack = [{} for _ in range(seq_len)] # Stores previous state for backtracking

    # Initialize the first position in our DP table
    for state in hmm_states:
        dp_table[0][state] = start_probs[state] * emit_probs[state][dna_seq[0]]
    
    # Main Viterbi logic: Fill DP table by finding most likely previous state
    for pos in range(1, seq_len):
        for current in hmm_states:
            best_prob = 0.0
            best_prev = None
            for previous in hmm_states:
                # Calculate new probability for this transition and emission
                new_prob = dp_table[pos-1][previous] * trans_probs[previous][current] * emit_probs[current][dna_seq[pos]]
                if new_prob > best_prob:
                    best_prob = new_prob
                    best_prev = previous
            dp_table[pos][current] = best_prob
            backtrack[pos][current] = best_prev
    
    # Traceback: Reconstruct the path by following backpointers
    final_state = max(dp_table[seq_len-1], key=dp_table[seq_len-1].get)
    best_path = [final_state]
    for pos in range(seq_len-1, 0, -1):
        final_state = backtrack[pos][final_state]
        best_path.insert(0, final_state)

    return best_path

print(find_best_path("CTTCATGTGAAAGCAGACGTAAGTCA"))

['E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E']


## The Viterbi Algorithm Steps

1. **Initialization**: Set up the first column of our Viterbi matrix based on initial probabilities
2. **Recursion**: For each position in the sequence, calculate:
   - For each possible state, find the most likely previous state
   - Update our matrix with the new probability
   - Keep track of the "backpointers" to remember which path we took
3. **Termination**: Find the most likely final state
4. **Traceback**: Follow the backpointers to reconstruct the most likely state sequence

## Example From the Nature Primer
In the primer, they mention calculating the probability of a specific path "EEEEEEEEEEEEEEEEEE5IIIIIII" given the observation "CTTCATGTGAAAGCAGACGTAAGTCA". When we run through the math, we get a log probability of -41.22.

Here's how we'd calculate this by hand:
1. For each position, multiply the emission probability with the transition probability
2. Convert to log space to avoid underflow
3. Sum all the log probabilities

For example, if we're in state E and see nucleotide C, we use log(P(C|E)). Then to move to another state E, we add log(P(E→E)).

## Implementation Notes

To implement the Viterbi algorithm, I'd need to:

1. Define all my states (E, I, 5)
2. Set up transition probabilities between states
3. Set up emission probabilities for each nucleotide from each state
4. Create a function to calculate the probability of a given path
5. Create the actual Viterbi algorithm to find the most likely path

The interesting thing is that depending on our parameters, the most likely path might actually just be all exons (EEEE...). This is actually a common outcome when transition probabilities heavily favor staying in one state. It doesn't mean our implementation is wrong - it's just what the math says given our parameters!

## Interesting Notes
- The most likely path might actually just be all exons (EEEE...) depending on our model parameters - this is totally fine!
- This algorithm is used all over the place - gene finding, speech recognition, and even in spell checking!

## Conclusion
The Viterbi algorithm is super useful for decoding the hidden structure in biological sequences. Once you implement it, you can tweak the parameters to better identify different genomic features like coding regions, introns, and splice sites. It's definitely one of the most important algorithms we've learned about in this class!